In [1]:
import pandas as pd
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///stocks_formatted.db')

In [3]:
df = pd.read_sql_table('price_close', con=engine)

In [8]:
df.drop(column='index', inplace= True)

TypeError: drop() got an unexpected keyword argument 'column'